In [ ]:
import json as json
import nba_functions as nba
import numpy as np


In [ ]:
game1 = json.load(open('warriors_game1.json'))

In [ ]:
nba.positional_smoothing(game1)
nba.velo_accel_smoothing(game1)
nba.get_masses(game1)
nba.velo_accel(game1)
nba.smooth_energy(game1)
nba.ball_flagger(game1)
nba.player_flagger(game1)
nba.player_flagger2(game1)

In [ ]:
game = '2017/10/17/HOUGSW/'

x = [] #time (in minutes)
y = [] #total energy
c = []
names = []
xo = []
xd = []
yo = []
yd = []
x2 = []
y2 = []
names2 = []
c2 = []
x2d = []
y2d = []
y2o = []
xo2 = []
xd2 = []


for team in ['homeplayers','awayplayers']:
    
    for key in game1[game][team].keys():
        
        has_ball = game1[game][team][key]['poss'][4:-4]
        defense = ~has_ball
        oc = np.array(game1[game][team][key]['on_court'])
        en = np.array(game1[game][team][key]['work'])
        en_no_nan = en[en==en]           
        en_off = en[has_ball]
        en_off = en_off[en_off==en_off]
        en_def = en[defense]
        en_def = en_def[en_def==en_def]
        
        if team == 'homeplayers':
            c.append(200)
            energy = np.nansum(en_no_nan[en_no_nan<10000])
            energyO = np.nansum(en_off[en_off<10000])
            energyD = np.nansum(en_def[en_def<10000])
            
            timeplayed = len(oc[oc==False])/25/60
            t_off = len(en_off)/25/60
            t_def = len(en_def)/25/60
            y.append(energy/(10**6))
            yo.append(energyO/(10**6))
            yd.append(energyD/(10**6))
            x.append(timeplayed)
            xo.append(t_off)
            xd.append(t_def)
            names.append(str(key))
         
        else:
            c2.append(0)
          
            energy = np.nansum(en_no_nan[en_no_nan<10000])
            energyO = np.nansum(en_off[en_off<10000])
            energyD = np.nansum(en_def[en_def<10000])

            timeplayed = len(oc[oc==False])/25/60
            t_off = len(en_off)/25/60
            t_def = len(en_def)/25/60
            
            y2.append(energy/(10**6))
            y2o.append(energyO/(10**6))
            y2d.append(energyD/(10**6))
            
            x2.append(timeplayed)
            xo2.append(t_off)
            xd2.append(t_def)
            names2.append(str(key))    

In [ ]:
from bokeh.io import show, reset_output, output_file
from bokeh.models import ColumnDataSource, HoverTool, LabelSet
from bokeh.plotting import figure


reset_output()

album_names = names
x = np.array(x)
energy = np.array(y)
energyO = np.array(yo)
energyD = np.array(yd)

radii =  np.ones(len(x))
size = 25*np.ones(len(x))

colors = [
    "#%02x%02x%02x" % (int(g), int(g), int(g)) for r, g in zip(c,c)
]

album_names2 = names2
energy2 = np.array(y2)
energyO2 = np.array(y2o)
energyD2 = np.array(y2d)
x2 = np.array(x2)
energy2 = np.array(y2)
radii2 =  np.ones(len(x2))
size2 = np.ones(len(x2))*25
colors2 = [
    "#%02x%02x%02x" % (int(g), int(g), int(g)) for r, g in zip(c2,c2)
]

source = ColumnDataSource(data=dict(x=x,
                                    xd=xd,
                                    xo=xo,
                                    energy=energy,
                                    energyO=energyO,
                                    energyD=energyD,
                                    names=album_names,
                                    radii = radii,
                                    size = size,
                                    colors = colors))

source2 = ColumnDataSource(data=dict(x=x2,
                                    xd=xd2,
                                    xo=xo2,
                                    energy=energy2,
                                    energyO=energyO2,
                                    energyD=energyD2,
                                    names=album_names2,
                                    radii = radii2,
                                    size = size2,
                                    colors = colors2))

p = figure(title = 'Total Energy Expended (Un-normalized)', tools=[HoverTool(tooltips=[('Player Name', '@names'),
                                      ('Minutes Played', '@x'),
                                      ('Total Work (MJ)', '@energy')])])

for i,srce in enumerate([source,source2]):
    
    teams = ['Warriors','Rockets']
    p.scatter(x='x', y='energy', radius='radii', fill_color = 'colors',source=srce,legend='%s' %teams[i])
    p.scatter(x='xo', y='energyO', marker = 'triangle', size = 'size', fill_color = 'colors',source=srce,legend='Offense %s' %teams[i])
    p.scatter(x='xd', y='energyD', marker = 'square', size = 'size', fill_color = 'colors',source=srce,legend='Defense %s' %teams[i])


p.legend.location = 'top_left'
p.legend.click_policy = 'hide'

p.xaxis.axis_label = 'Minutes Played'
p.yaxis.axis_label = r'Energy (MJ)'
# p.title.axis_label = 'test'

output_file("warriors_rockets_game1.html", title="Total Energy Expended (Un-normalized)")

show(p)

In [ ]:
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure

reset_output()
output_file("panning.html")

x = list(range(11))
y0 = x
y1 = [10-xx for xx in x]
y2 = [abs(xx-5) for xx in x]

# create a new plot
s1 = figure(plot_width=250, plot_height=250, title=None)
s1.circle(x, y0, size=10, color="navy", alpha=0.5)

# create a new plot and share both ranges
s2 = figure(plot_width=250, plot_height=250, x_range=s1.x_range, y_range=s1.y_range, title=None)
s2.triangle(x, y1, size=10, color="firebrick", alpha=0.5)

# create a new plot and share only one range
s3 = figure(plot_width=250, plot_height=250, x_range=s1.x_range, title=None)
s3.square(x, y2, size=10, color="olive", alpha=0.5)

p = gridplot([[s1, s2, s3]])

# show the results
show(p)

In [ ]:
game = '2017/10/17/HOUGSW/'

x = [] #time (in minutes)
y = [] #total energy
c = []
names = []

x2 = []
y2 = []
names2 = []
c2 = []



for team in ['homeplayers','awayplayers']:
    
    for key in game1[game][team].keys():
        
        has_ball = game1[game][team][key]['poss'][4:-4]
        defense = ~has_ball
        oc = np.array(game1[game][team][key]['on_court'])
        en = np.array(game1[game][team][key]['work'])
        en_no_nan = en[en==en]           
        
        
        if team == 'homeplayers':
            c.append(200)
            energy = np.nansum(en_no_nan[en_no_nan<10000])
            
            
            timeplayed = len(oc[oc==False])/25/60
            
            y.append(energy/(10**6))
            
            x.append(timeplayed)
          
            names.append(str(key))
         
        else:
            c2.append(0)
          
            energy = np.nansum(en_no_nan[en_no_nan<10000])
            

            timeplayed = len(oc[oc==False])/25/60
          
            y2.append(energy/(10**6))
           
            
            x2.append(timeplayed)
            names2.append(str(key))    